In [2]:
#https://pythonprogramming.net/change-show-new-frame-tkinter/
#part 29 is conversion to executable!! 

import tkinter as tk #lower case in Py3
from tkinter import ttk #kinda like CSS for tkinter
#get more rounded buttons isntead of ugly square ones (post 2000 look). change all buttons

In [3]:
import matplotlib 
from matplotlib import style

In [4]:
matplotlib.use('TkAgg')

In [5]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
#figure canvas allows you graph on canvas
#toolbar allows zooming etc
from matplotlib.figure import Figure
#below to redraw graph every xseconds
import matplotlib.animation as animation
from matplotlib import style #available styles C:\Users\miste\Anaconda3\Lib\site-packages\matplotlib\mpl-data\stylelib


In [6]:
#Tkinter page8
import urllib
import json
import pandas as pd
import numpy as np

In [7]:
from matplotlib import pyplot as plt

In [ ]:
LARGE_FONT = ("Verdana", 12) #size `12
NORM_FONT = ("Verdana", 10)
SMALL_FONT = ("Verdana", 8)

style.use("ggplot")  #changes from ugly blue to nicer red
##available styles C:\Users\miste\Anaconda3\Lib\site-packages\matplotlib\mpl-data\stylelib


f = Figure(figsize = (5,5), dpi = 100)
a = f.add_subplot(111) #111 means only one chart

exchange = 'BTC-e'
DatCounter = 9000 #forces update 
programName = 'btce'

resampleSize = '15Min'
DataPace = '1d' #size of data set
candleWidth = 0.008 #default witdth

topIndicator = 'none'
bottomIndicator = 'none'
middleIndicator = 'none'

EMAs = [] #can have multiple else erases
SMAs = []


def changeTimeFrame(tf):  #Timeframe and sample size option
    global DataPace
    #global DatCounter
    if DataPace == '7d' and resampleSize == '1Min':  
        popupmsg('Too many much data, choose smaller time frame or higher OHLC sample')
        #we will need more later..like 6months *15mins.   
    else:
        DataPace = tf #timeframe
        DatCounter = 9000  
        
def changeSampleSize(size, width):  #width of candlestick
    
    global resampleSize 
    global candleWidth
    if tf == '7d' and resampleSize == '1Min':  
        popupmsg('Too many much data, choose smaller time frame or higher OHLC sample')
    elif DataPace == 'tick': 
        popupmsg('you are currently viewing tick data, not OHLC.')
    else:
        resampleSize = size
        DatCounter = 9000
        candleWidth = width
        
def changeExchange(toWhat, pn):  
    global exchange
    global DatCounter
    global programName #global allows modifitcation
    exchange = toWhat
    programName = pn
    DatConter = 9000 #force update
    
    
def popupmsg(msg): #mini instance of tkinter
    popup = tk.TK()
    
    
    popup.wm_title('!')
    label = ttk.Label(popup, text = msg, font = NORM_FONT)
    label.pack(side = "top", fill = 'x', pady = 10) #pads top and bottom, x would be edges
    B1 = ttk.Button(popup, text = 'Oka', command = leavemini) #not defined yet
    B1.pack()
    popup.mainloop()
    
    
    
#def animate(): TypeError: animate() takes 0 positional arguments but 1 was given
def animate(i):
    
    '''#soon pull as json data instead of files
    #important: file below needs to be on same directory as script,search for this notebook on drive 
    pullData = open('sampleData.txt',"r").read() #open with intention to read = 'r'
    dataList = pullData.split('\n') #by new line
    xList = []
    yList = []
    for eachLine in dataList:
        if len(eachLine)> 1: #skips empty ones
            x, y = eachLine.split(',')
            xList.append(int(x))
            yList.append(int(y))
    a.clear() #gets rid of everything on subplot so it deosnt get messy, take up more RAM etc
    a.plot(xList, yList)  '''
    
    #dataLink = 'https://btc-e.com/api/3/trades/btc_usd?limit = 2000'#was json format, unit time stamp at end
    dataLink = 'https://api.bitfinex.com/v1/trades/BTCUSD?limit_trades=2000' #? adds parameter
    #"timestamp":1511872455,"tid":102324136,"price":"9910.4","amount":"0.6","exchange":"bitfinex","type":"sell"}
    
    data = urllib.request.urlopen(dataLink)  
    #decode bytes to strings
    
    data = data.read().decode('utf-8') #data = data.readall().decode("utf-8")  AttributeError: 'HTTPResponse' object has no attribute 'readall'
    pd.options.mode.chained_assignment = None
    
    #now load json string
    data = json.loads(data) #data = data['btc_usd'] #TypeError: list indices must be integers or slices, not str
    #now crunch the 2000 rows with pandas
    data = pd.DataFrame(data)
    data.head()
    
    buys = data[(data['type']=='buy')]#buys = data[(data['type']=='bid')]  #dataframe too
    #convert unix time stamp -matplotlib doesnt recognize it, using nupy
    buys['datestamp'] = np.array(buys['timestamp']).astype('datetime64[s]') #add column
    buyDates = (buys['datestamp']).tolist() #list of converted numpy timestamps..the x's
    
    sells = data[(data['type'])=='sell']
    sells['datestamp'] = np.array(sells['timestamp']).astype('datetime64[s]') #add column
    sellDates = (sells['datestamp']).tolist()
    
    a.clear()
    
    a.plot_date(buyDates, buys['price'], 'b', label = 'buy') #x's and y's
    a.plot_date(sellDates, sells['price'], 'r', label = 'sell')
    
    a.legend(bbox_to_anchor = (0, 1.02, 1, .102), loc = 3, ncol = 2, 
             borderaxespad = 0) #check matplotlib tutorial for description
    title = 'BTCUSD Prices'#\nLast Price: '+str(data["price"][1999])
    a.set_title(title)
    


In [ ]:
class SeaofBTCapp(tk.Tk): #inheritenign from tk class
    
    def __init__(self, *args, **kwargs):  #arguments are anything, keyword arguments are dictionaries
    
        tk.Tk.__init__(self, *args, **kwargs)  #init tk too
        #this applicaiton is a container that we will populate
        
        #tk.Tk.iconbitmap(self, defaut = 'clientapp.ico')#change icon
        tk.Tk.wm_title(self, 'SeaOfTBTCApp') #change wintodw title
        
        container = tk.Frame(self) #frame is a part of tk, it's a 'window'
        #tk gives 2 options: pack stuff in or grid
        
        container.pack(side="top", fill = "both", expand = True) #fill will fill space , expand will exp beyond space in window
        container.grid_rowconfigure(0, weight = 1) #0 sets min size, weight gives 'priority'
        container.grid_columnconfigure(0, weight = 1)
        
        menubar = tk.Menu(container) #menu inside container
        filemenu = tk.Menu(menubar, tearoff = 0) #tearoff allows File, Edit to become their own little windows, for upcoming Trading option
        filemenu.add_command(label = 'Save settings', 
                             command = lambda: popupmsg('not supported yet!'))#adds option to menu, eventually save api info wh
        filemenu.add_separator() #adds a divider between options
        filemenu.add_command(label = 'Exit', command = quit)
        menubar.add_cascade(label = 'File', menu  = filemenu)
        
        
        #=========================
        exchangeChoice = tk.Menu(menubar, tearoff = 1)
        #ukraine, china, GB
        exchangeChoice.add_command(label ='BTC-e', 
                                  command = lambda: changeExchange('BTC-e', 'btce')) #this fucntion doesnt exist yet
        exchangeChoice.add_command(label ='BitFinex', 
                                  command = lambda: changeExchange('Bitfinex', 'bitfinex')) #this fucntion doesnt exist yet
        exchangeChoice.add_command(label ='Bitstamp', 
                                  command = lambda: changeExchange('Bitstamp', 'bitstamp')) #this fucntion doesnt exist yet
        
        
        menubar.add_cascade(label = 'Exchange', menu  = exchangeChoice)
        #==========================
        
        dataTF = tk.Menu(menubar, tearoff = 1)
        dataTF.add_command(label = 'Tick',
                           command = lambda: changeTimeFrame('tick')) #new function
        dataTF.add_command(label = '1 Day',
                           command = lambda: changeTimeFrame('1d')) #new function
        dataTF.add_command(label = '1 Week',
                           command = lambda: changeTimeFrame('7d')) #new function
        
        menubar.add_cascade(label = 'Data TimeFrame', menu  = dataTF)
        #============================
        OHLCI = tk.Menu(menubar, tearoff = 1)
        OHLCI.add_command(                   
                          command = lambda: changeTimeFrame('tick')) 
        dataTF.add_command(label = '1 minute',
                           command = lambda: changeTimeFrame('1Min', 0.0005)) #2nd param width of candlestick
        dataTF.add_command(label = '5 minute',
                           command = lambda: changeTimeFrame('5Min', 0.0003))
        dataTF.add_command(label = '15 minute',
                           command = lambda: changeTimeFrame('15Min', 0.0008))
        dataTF.add_command(label = '1 Hour',
                           command = lambda: changeTimeFrame('1H', 0.0016))
        
        menubar.add_cascade(label = 'OHLC Interval', menu = OHLCI)
    
        #=======
        mainI = tk.Menu(menubar, tearoff = 1)
        mainI.add_comand(label = 'None',
                           command = lambda: addMiddleIndicator('None')
        mainI.add_comand(label = 'SMA'
                           command = lambda: addMiddleIndicator('sma')
        mainI.add_comand(label = 'EMA'
                           command = lambda: addMiddleIndicator('ema')
                         
        menubar.add_cascade(label = 'Main/middle', menu = mainI)
        
        topIndi = tk.Menu(menubar, tearoff = 1)
        topIndi.add_comand(label = 'None',
                           command = lambda: addTopIndicator('None')
        topIndi.add_comand(label = 'RSI'
                           command = lambda: addTopIndicator('rsi')
        topIndi.add_comand(label = 'MACD'
                           command = lambda: addTopIndicator('macd')
        menubar.add_cascade(label = 'Top Indicator', menu = topIndi)
        
        topIndi = tk.Menu(menubar, tearoff = 1)
        topIndi.add_comand(label = 'None',
                           command = lambda: addTopIndicator('None')
        topIndi.add_comand(label = 'RSI'
                           command = lambda: addTopIndicator('rsi')
        topIndi.add_comand(label = 'MACD
                           command = lambda: addTopIndicator('macd')
        menubar.add_cascade(label = 'Top Indicator', menu = topIndi)
                                          
        #============================
        
        
        tk.Tk.config(self, menu = menubar)
        
        self.frames = {} #empty dict
        
        #frame = StartPage(container, self) #doesnt exist yet but define soon
        #self.frames[StartPage] = frame
    
        #need this so that pages show up
        for F in (StartPage, BTCe_Page): #pass tuple of pages
            
            #wrong:  frame = StartPage(container, self)
            frame = F(container, self)
            
            self.frames[F] = frame
            
            frame.grid(row= 0, column = 0, sticky = "nsew") #why grid is superior to pack: assigns some frame to grid, north south east west stretch evverythign
        
        self.show_frame(StartPage) #doesnt esxist yet eitehr
        
    def show_frame(self, cont): #controller
        
        frame = self.frames[cont]  #key 
        frame.tkraise() #raises to front,  inherited this method from tk above
        
#def qf():
   # print('example function for button below')
    
######################
#these 2 following classes should look nearly identitcal, just diff buttons

class StartPage(tk.Frame):  #inherit all stuff from frame
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent) #parent class is seaofbtcapp
    
        label = tk.Label(self, text = "Alpha Bitcon trading app", font = LARGE_FONT) #return object to label, now do smeothign to it
        label.pack(pady = 10, padx = 10) #pads top and bottom
        
        #all buttons changed from tk.Button to ttk.Button for style
        button1 = ttk.Button(self, text = "Agree", 
                           command = lambda: controller.show_frame(BTCe_Page)) #button object returned from tk.Button()..qf is defined above
        button1.pack()
        
        
        button2 = ttk.Button(self, text = "Disagree", 
                            command = quit) #button object returned from tk.Button()..qf is defined above
        button2.pack()



        
########################   

class BTCe_Page(tk.Frame): #graph page. need to add canvas and then graph
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = tk.Label(self, text = "Graph Page", font = LARGE_FONT) #return object to label, now do smeothign to it
        label.pack(pady = 10, padx = 10) #pads top and bottom
        
        button1 = ttk.Button(self, text = "Back to Home", 
                            command = lambda: controller.show_frame(StartPage)) #button object returned from tk.Button()..qf is defined above
        button1.pack()
                            

        ########graph beneath 
        #f = Figure(figsize = (5,5), dpi = 100)
        #a = f.add_subplot(111) #111 means only one chart
        #a.plot([1,2,3,4,5], [
        5,6, 1, 3, 4]) #x list and y list..this is now in backgrou. need to bring in front
        
        #this brings graph from back to front
        canvas = FigureCanvasTkAgg(f, self)
        canvas.show()
        canvas.get_tk_widget().pack(side = tk.TOP, fill = tk.BOTH, expand = True) #packing canvas into tkinger
        #above has no navigation bar..so
        toolbar = NavigationToolbar2TkAgg(canvas, self)
        toolbar.update()
        canvas._tkcanvas.pack(side = tk.TOP, fill = tk.BOTH, expand = True)
        
        
app = SeaofBTCapp()
app.geometry('1280x720') #or 800x600 unless u are on a phone lol
ani = animation.FuncAnimation(f,animate, interval = 5000)#f is figure, anim, interval in millseconds..5 to slow it down a bit
app.mainloop()

In [39]:
?animation.FuncAnimation

In [15]:
import sys

In [16]:
sys.path

['',
 'C:\\Users\\miste\\Anaconda3\\python36.zip',
 'C:\\Users\\miste\\Anaconda3\\DLLs',
 'C:\\Users\\miste\\Anaconda3\\lib',
 'C:\\Users\\miste\\Anaconda3',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages\\Sphinx-1.5.6-py3.6.egg',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages\\setuptools-27.2.0-py3.6.egg',
 'C:\\Users\\miste\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\miste\\.ipython']

In [60]:
# ING hedging
class INGHedging(tk.Tk): #inheritenign from tk class
    
    def __init__(self, *args, **kwargs):  #arguments are anything, keyword arguments are dictionaries
    
        tk.Tk.__init__(self, *args, **kwargs)  #init tk too
        #this applicaiton is a container that we will populate
        
        #tk.Tk.iconbitmap(self, defaut = 'clientapp.ico')#change icon
        tk.Tk.wm_title(self, 's') #change wintodw title
        
        container = tk.Frame(self) #frame is a part of tk, it's a 'window'
        #tk gives 2 options: pack stuff in or grid
        container.pack(side="top", fill = "both", expand = True) #fill will fill space , expand will exp beyond space in window
        container.grid_rowconfigure(0, weight = 1) #0 sets min size, weight gives 'priority'
        container.grid_columnconfigure(0, weight = 1)
        
        self.frames = {} #empty dict
        
        #frame = StartPage(container, self) #doesnt exist yet but define soon
        #self.frames[StartPage] = frame
    
        #need this so that pages show up
        for F in (StartPage, PageOne, PageTwo, PageThree): #pass tuple of pages
            
            #wrong:  frame = StartPage(container, self)
            frame = F(container, self)
            
            self.frames[F] = frame
            
            frame.grid(row= 0, column = 0, sticky = "nsew") #why grid is superior to pack: assigns some frame to grid, north south east west stretch evverythign
        
        self.show_frame(StartPage) #doesnt esxist yet eitehr
        
    def show_frame(self, cont): #controller
        
        frame = self.frames[cont]  #key 
        frame.tkraise() #raises to front,  inherited this method from tk above
        
#def qf():
   # print('example function for button below')
    
######################
#these 2 following classes should look nearly identitcal, just diff buttons
class StartPage(tk.Frame):  #inherit all stuff from frame
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent) #parent class is seaofbtcapp
    
        label = tk.Label(self, text = "ING Hedging Strategies", font = LARGE_FONT) #return object to label, now do smeothign to it
        label.pack(pady = 10, padx = 10) #pads top and bottom
        
        #all buttons changed from tk.Button to ttk.Button for style
        button = ttk.Button(self, text = "Cylinder", 
                           command = lambda: controller.show_frame(PageOne)) #button object returned from tk.Button()..qf is defined above
        button.pack()
        
        button1 = ttk.Button(self, text = "Participating Forward", 
                            command = lambda: controller.show_frame(PageTwo)) #button object returned from tk.Button()..qf is defined above
        button1.pack()
        
        button2 = ttk.Button(self, text = "Forward Extra", 
                            command = lambda: controller.show_frame(PageThree)) #button object returned from tk.Button()..qf is defined above
        button2.pack()
        
        button3 = ttk.Button(self, text = "Graph Page_Inverse Forward Extra", 
                            command = lambda: controller.show_frame(PageThree)) #button object returned from tk.Button()..qf is defined above
        button3.pack()
        
        
        
        
        
########################        
class PageOne(tk.Frame): #creating page one above, inheriting from tk.Frame
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text = "Cylinder", font = LARGE_FONT) #return object to label, now do smeothign to it
        label.pack(pady = 10, padx = 10) #pads top and bottom
        
        button1 = ttk.Button(self, text = "Back to Home", 
                            command = lambda: controller.show_frame(StartPage)) #button object returned from tk.Button()..qf is defined above
        button1.pack()
        
        button2 = ttk.Button(self, text = "Page 2", 
                            command = lambda: controller.show_frame(PageTwo)) #button object returned from tk.Button()..qf is defined above
        #button1.pack()
        button2.pack()
        
        button3 = ttk.Button(self, text = "Page 3", 
                            command = lambda: controller.show_frame(PageTwo)) #button object returned from tk.Button()..qf is defined above
        button3.pack()
                   
########################   

class PageTwo(tk.Frame):
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = tk.Label(self, text = "Participating Forward", font = LARGE_FONT) #return object to label, now do smeothign to it
        label.pack(pady = 10, padx = 10) #pads top and bottom
        
        button1 = ttk.Button(self, text = "Back to Home", 
                            command = lambda: controller.show_frame(StartPage)) #button object returned from tk.Button()..qf is defined above
        button1.pack()
                            
        button2 = ttk.Button(self, text = "Page One", 
                            command = lambda: controller.show_frame(PageOne)) #button object returned from tk.Button()..qf is defined above
        button2.pack()
        
########################   

class PageThree(tk.Frame): #graph page. need to add canvas and then graph
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = tk.Label(self, text = "Forward Extra", font = LARGE_FONT) #return object to label, now do smeothign to it
        label.pack(pady = 10, padx = 10) #pads top and bottom
        
        button1 = ttk.Button(self, text = "Back to Home", 
                            command = lambda: controller.show_frame(StartPage)) #button object returned from tk.Button()..qf is defined above
        button1.pack()
                            
        button2 = ttk.Button(self, text = "Page One", 
                            command = lambda: controller.show_frame(PageOne)) #button object returned from tk.Button()..qf is defined above
        button2.pack()
        ########graph beneath 
        #f = Figure(figsize = (5,5), dpi = 100)
        #a = f.add_subplot(111) #111 means only one chart
        #a.plot([1,2,3,4,5], [5,6, 1, 3, 4]) #x list and y list..this is now in backgrou. need to bring in front
        
        #this brings graph from back to front
        canvas = FigureCanvasTkAgg(f, self)
        canvas.show()
        canvas.get_tk_widget().pack(side = tk.TOP, fill = tk.BOTH, expand = True) #packing canvas into tkinger
        #above has no navigation bar..so
        toolbar = NavigationToolbar2TkAgg(canvas, self)
        toolbar.update()
        canvas._tkcanvas.pack(side = tk.TOP, fill = tk.BOTH, expand = True)
        
        
app = INGHedging()
ani = animation.FuncAnimation(f,animate, interval = 1000)#f is figure, anim, interval in millseconds
app.mainloop()

In [54]:
salaryRange = []
salaryStart = 20000
for i in range(1,5):

    salaryRange.append(salaryStart*i)
print(salaryRange)

[20000, 40000, 60000, 80000]


In [55]:
workers = 1400
for salary in salaryRange:
    print(workers * salary) 

28000000
56000000
84000000
112000000


In [56]:
50 *3

150